# Test 客规Insert
This notebook replicates the functionality of the `test_ollama.py` script.

In [3]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import gpt_4o_mini_complete, gpt_4o_complete, openai_embed
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import openai_complete_if_cache
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import numpy as np


# Uncomment the below two lines if running in a jupyter notebook to handle the async nature of rag.insert()
import nest_asyncio
nest_asyncio.apply()

In [7]:
WORKING_DIR = "./KG"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

# 在导入部分添加
from dotenv import load_dotenv

# 在代码的开头加载环境变量
load_dotenv()

True

In [5]:
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs
) -> str:
    return await openai_complete_if_cache(
        "Qwen/Qwen2.5-7B-Instruct",
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        base_url="https://api.siliconflow.cn/v1/",
        **kwargs,
    )


async def embedding_func(texts: list[str]) -> np.ndarray:
    return await siliconcloud_embedding(
        texts,
        model="netease-youdao/bce-embedding-base_v1",
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        max_token_size=512,
    )



In [8]:
# function test
async def test_funcs():
    result = await llm_model_func("How are you?")
    print("llm_model_func: ", result)

    result = await embedding_func(["How are you?"])
    print("embedding_func: ", result)


asyncio.run(test_funcs())


llm_model_func:  I'm doing well, thank you for asking! How can I assist you today?
embedding_func:  [[ 3.82568017e-02  1.12520000e-02 -1.56639684e-02  1.11631686e-02
   9.46056377e-03 -5.98626165e-03  9.40134283e-03 -9.31251049e-03
  -1.66411158e-02 -1.13507025e-02 -5.73654622e-02 -9.91459191e-03
   8.42518210e-02 -7.01275980e-03 -2.97092292e-02  8.16757046e-03
   8.60185828e-03 -3.19793709e-02  1.01045927e-03  9.37173143e-03
   7.17364401e-02 -1.39860390e-02  2.33725756e-02  3.42495106e-02
   3.63617279e-02 -4.36508516e-03 -5.16703678e-03 -2.17760753e-04
  -2.05546408e-03 -9.23354924e-03 -2.51294672e-02  1.53086428e-02
   1.96317788e-02 -7.91588128e-02 -3.04593630e-02 -3.39040533e-03
  -4.94298413e-02 -1.93159338e-02  1.10703893e-01  8.81209299e-02
   2.80312989e-02  3.66380922e-02  2.40042675e-02 -3.75264063e-02
  -3.78619954e-02  9.24095220e-04  9.65796690e-03 -2.00364571e-02
   2.73009054e-02  2.68962281e-03  5.78639039e-04  2.07668506e-02
  -1.32260355e-03  2.37476435e-02 -6.85385

In [9]:
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func,
    embedding_func=EmbeddingFunc(
        embedding_dim=768, max_token_size=512, func=embedding_func
    ),
)


/Users/llp/miniconda3/envs/lightrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './KG/vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './KG/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './KG/vdb_chunks.json'} 0 data
INFO:lightrag:Loaded document status storage with 0 records


In [3]:
# Choose the model here
model_choice = "ollama"  # Options: "ollama", "openai"

llm_model_func = ollama_model_complete
embedding_func = EmbeddingFunc(
    embedding_dim=1024,
    max_token_size=8192,
    func=lambda texts: ollama_embed(
        texts,
        embed_model="bge-m3:latest"
    )
)
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func, 
    llm_model_name='qwen2.5:7b-instruct-q4_K_M',
    embedding_func=embedding_func,
    llm_model_max_token_size=32768,
    llm_model_kwargs={"host": "http://localhost:11434", "options": {"num_ctx": 32768}},

)

/Users/llp/miniforge3/envs/lightrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': './dickens/vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': './dickens/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': './dickens/vdb_chunks.json'} 0 data
INFO:lightrag:Loaded document status storage with 0 records


In [10]:
with open("./kg1.txt") as f:
    rag.insert(f.read())

INFO:lightrag:Inserting 1 to doc_status
INFO:lightrag:Stored 1 new unique documents
INFO:lightrag:Number of batches to process: 1.
INFO:lightrag:Start processing batch 1 of 1.
INFO:lightrag:Inserting 1 to doc_status
INFO:lightrag:Inserting 8 to chunks
INFO:lightrag:Inserting 1 to full_docs
INFO:lightrag:Inserting 8 to text_chunks
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)

In [14]:
# Perform naive search
print("naive.....................")
print(rag.query("对比一级应急响应和四级应急响应中，广州南、深圳北、长沙南、广州、广州东、长沙、深圳站启动应急响应的标准？", param=QueryParam(mode="naive")))

INFO:lightrag:Non-embedding cached missed(mode:naive type:query)


naive.....................


INFO:lightrag:Inserting 1 to llm_response_cache


根据提供的文档信息，广州南、深圳北、长沙南、广州、广州东、长沙、深圳站启动应急响应的标准如下：

### 一级应急响应启动标准
- 广州南、广州、广州东站旅客滞留人数达到或预计达到一、二级应急响应标准时。

### 四级应急响应启动标准
- 集团公司认为需要按四级应急响应启动时。
- 其他省会城市、地市级主要车站预计3小时内滞留旅客达到车站正常最大候车能力90%，需要启用站内或车站周边集结地候车时。

总结来说，广州南、深圳北、长沙南、广州、广州东、长沙、深圳站启动一级应急响应的标准主要是这些站的旅客滞留人数达到高预警水平；而启动四级应急响应则是由集团公司根据实际情况决定，通常针对其他规模较大的车站的预计滞留情况。


In [6]:
# Perform local search
print("local.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="local")))

INFO:lightrag:Non-embedding cached missed(mode:local type:query)
INFO:lightrag:Non-embedding cached missed(mode:local type:keywords)


local.....................


INFO:lightrag:Query nodes: Plot elements, Character development, Setting, Conflict, top_k: 60, cosine: 0.2
INFO:lightrag:Local query uses 60 entites, 65 relations, 0 chunks


The story "A Christmas Carol" by Charles Dickens explores several key themes that resonate throughout the narrative:

1. **Redemption and Transformation**: The central theme revolves around Scrooge's transformation from a miserly, bitter old man to a kind, generous person who values family and charity. This transformation is facilitated by his encounters with the three spirits: the Ghost of Christmas Past, Present, and Yet to Come.

2. **Moral Guidance Through Visions**: The spirits guide Scrooge through his past, present, and future, offering moral lessons that challenge his behavior and influence his character development. These visions reveal the consequences of his actions and the importance of change.

3. **The Spirit of Christmas**: The story emphasizes the true spirit of Christmas, which includes giving, charity, and family bonds. Scrooge's experiences during the visits from the spirits highlight these values and their impact on society.

4. **Social Class and Inequality**: Dick

In [8]:
# Perform global search
print("global.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="global")))

INFO:lightrag:Non-embedding cached hit(mode:global type:query)


global.....................
The story of "A Christmas Carol" by Charles Dickens is rich with thematic elements that resonate deeply within its narrative. The primary themes include transformation, redemption, and the spirit of Christmas.

### Transformation and Redemption
One of the most significant themes in "A Christmas Carol" is the profound personal transformation experienced by Ebenezer Scrooge. Initially a bitter, miserly old man who despises Christmas and his fellow human beings, Scrooge undergoes a dramatic change after being visited by the three spirits—ghosts of Christmas Past, Present, and Yet to Come. These spirits reveal to him not only his past mistakes but also the dire consequences of his current behavior. Through this journey, Scrooge comes to understand the value of generosity, compassion, and family, ultimately leading to a complete transformation from a miser to a kind and generous man.

### The Spirit of Christmas
The spirit of Christmas is another crucial theme th

In [9]:
# Perform hybrid search
print("hybrid.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="hybrid")))

INFO:lightrag:Non-embedding cached missed(mode:hybrid type:query)
INFO:lightrag:Non-embedding cached missed(mode:hybrid type:keywords)


hybrid.....................


INFO:lightrag:Query nodes: Narrative elements, Plot development, Character arcs, Symbolism, top_k: 60, cosine: 0.2
INFO:lightrag:Query edges: Top themes, Story analysis, top_k: 60, cosine: 0.2
INFO:lightrag:Global query uses 62 entites, 60 relations, 0 chunks
INFO:lightrag:Local query uses 60 entites, 66 relations, 0 chunks


The story of "A Christmas Carol" by Charles Dickens is rich with several prominent themes that resonate throughout the narrative:

1. **Redemption and Transformation**: One of the most central themes is the transformative journey of Ebenezer Scrooge. Initially portrayed as a bitter, miserly old man, Scrooge undergoes a significant change after being visited by the three spirits of Christmas past, present, and yet to come. These visits reveal his misdeeds and the potential consequences if he does not alter his ways.

2. **The Spirit of Christmas**: The novel emphasizes the true spirit of Christmas, which transcends mere materialism and commercialization. Scrooge learns that generosity, compassion, and kindness are essential virtues that should be practiced throughout the year, not just during the festive season.

3. **Class Differences and Social Inequality**: The story starkly contrasts the wealthy Scrooge with his poor employees and neighbors, such as Bob Cratchit and his family. This

In [15]:
# Perform mix search (Knowledge Graph + Vector Retrieval)
print("mix.....................")
print(rag.query("对比一级应急响应和四级应急响应中，广州南、深圳北、长沙南、广州、广州东、长沙、深圳站启动应急响应的标准？", param=QueryParam(mode="mix")))

INFO:lightrag:Non-embedding cached missed(mode:mix type:query)
INFO:lightrag:Non-embedding cached missed(mode:mix type:keywords)


mix.....................


INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Query nodes: 广州南站, 深圳北站, 长沙南站, 广州站, 广州东站, 长沙站, 深圳站, 一级响应, 四级响应, top_k: 60, cosine: 0.2
INFO:lightrag:Query edges: 应急响应, 紧急应对措施, 高铁站和火车站, 启动标准, top_k: 60, cosine: 0.2
INFO:lightrag:Local query uses 60 entites, 51 relations, 3 chunks
INFO:lightrag:Global query uses 80 entites, 60 relations, 3 chunks


### 一级应急响应启动标准

一级应急响应启动标准较为严格，具体如下：
1. 广州南、深圳北、长沙南、广州、广州东、长沙、深圳站预计4～6小时内或旅客列车运行秩序已明确受影响时段内滞留旅客达到车站正常和应急最大候车能力，需要属地地方政府协调组织旅客疏散、采取交通管制等措施的。
2. 集团公司认为需要启动一级应急响应的。

### 四级应急响应启动标准

四级应急响应启动标准为：
1. 广州南、深圳北、长沙南、广州、广州东、长沙、深圳站预计2小时内滞留旅客达到车站正常最大候车能力60%的，需要采取分段截留、分批进站等限制进站措施的。
2. 集团公司认为需要按四级应急响应启动的。

### 重点差异

- 启动一级应急响应的标准中达到旅客滞留达到车站正常最大候车能力的90%以上（达到车站正常和应急最大候车能力），需属地政府协调组织旅客疏散、采取交通管制等措施。
- 启动四级应急响应的标准则颗粒度更细且分级较小，仅需达到车站正常最大候车能力的60%，需采取分段截留、分批进站等限制进站措施。

### 概括

一级应急响应的启动条件更加严苛，不仅涵盖了滞留旅客更多数量的情况，还强调了需做出较大规模的应急处置措施，确保旅客可以迅速平安疏散交通管制等政府措施的实施。四级应急响应的条件覆盖了更早启动，满足上述条件则马上进入应急响应，虽然也需要采取措施，但相对操作细节更多。

---

### 参考信息
[DC] 总部根据实际情况提供详细的应急响应启动标准，明确了不同等级响应的具体联动措施要求。
[DC] 根据广州局集团的内部管理规定，针对不同等级的响应标准会采取不同的应急处理措施。
[DC] 在广州局集团公司突发事件总体应急预案中，明确了一级应急响应的标准，即车站滞留旅客达到正常最大候车能力的90%，必须实施交通管制等措施。
[DC] 一级应急响应的启动条件不仅包括滞留人数超过90%，还须有地方政府协调的应急措施启动，而四级应急响应启动条件则更为具体，需要先达到60%的候车能力，但同样效果上也会协调地方政府资源。
